# Fluxonium with multi-path coupling(Simultaneous X gate)

This example is based on Nguyen, L. B. et al. Blueprint for a High-Performance Fluxonium Quantum Processor. PRX Quantum 3, 037001 (2022).
We simulate a 6 fluxonium qubit system from an underlying periodic lattice.
Idling hamiltonian of the system is
$$ H(0) = \sum_i H_{\mathrm{f},i}  + \sum_{\langle i,j \rangle } H_{ij}. $$
Hamiltonian of single fluxonium is
$$ H_{\mathrm{f},i} = 4E_{\mathrm{C},i} n_i^2 + \frac{1}{2} E_{\mathrm{L},i} (\varphi_i +\varphi_{\mathrm{ext},i})^2
    -E_{\mathrm{J},i}\cos \left( \varphi_i \right).$$
The coupling terms have the form
$$H_{ij} = J_{\mathrm{C}} n_i n_j - J_{\mathrm{L}} \varphi_i \varphi_j.$$
The couplings are chosen in a way such that the idling $ZZ$-crosstalk is almost zero.

In [1]:
import jax
import jax.numpy as jnp
import haiku as hk

import supergrad
from supergrad.helper import Evolve
from supergrad.utils import compute_fidelity_with_1q_rotation_axis
from supergrad.utils.gates import sigmax
from supergrad.utils.optimize import scipy_minimize, adam_opt
from supergrad.utils.utility import tree_print
from supergrad.utils.fidelity import pauli_diagnose_partition

from supergrad.scgraph.graph_mpc_fluxonium_5x5_periodic import XGatePeriodicGraphOpt

truncated_dim = 3  # how many levels we keep after diagonalization
add_random = True  # whether to add deviations to fluxonium parameters ec,ej,el
share_params = True  # whether we treat marked parameters as shared when computing gradients
unify_coupling = True  # set all couplings to be equal
compensation_option = 'only_vz'  # Use virtual-z compensation in fidelity computation

# instance the quantum processor graph, and choose a subset for time evolution
gate_graph = XGatePeriodicGraphOpt(1)
qubit_subset = gate_graph.subgraph(['q02', 'q03', 'q12', 'q13', 'q22', 'q23'])

evo = Evolve(qubit_subset, truncated_dim, add_random, share_params, unify_coupling, compensation_option)
target_unitary = supergrad.tensor(*([sigmax()] * len(qubit_subset.nodes)))


Let us print the parameters stored in the evo instance. These parameters are parsed from gate_graph. For how we construct this graph, please check `examples/fluxonium_multipath_coupling/graph_5x5_periodic.py` and the instructions inside.

In [2]:
params = evo.all_params
tree_print(params)


{ 'capacitive_coupling_all_unify': {'strength': 0.07225663},
  'inductive_coupling_all_unify': {'strength': -0.01256637},
  'q02': {'ec': 6.3532393823, 'ej': 25.1103087675, 'el': 6.344787656},
  'q02_pulse_cos': { 'amp': 0.07467672,
                     'length': 39.82459069,
                     'omega_d': 3.70771137,
                     'phase': 0.7558433},
  'q03': {'ec': 6.2550610547000005, 'ej': 25.1411401038, 'el': 5.000964059},
  'q03_pulse_cos': { 'amp': 0.06839676,
                     'length': 39.81302627,
                     'omega_d': 2.56897537,
                     'phase': 0.85390569},
  'q12': {'ec': 6.2233456874, 'ej': 25.1512697752, 'el': 5.6012793457},
  'q12_pulse_cos': { 'amp': 0.07142831,
                     'length': 39.85070846,
                     'omega_d': 3.07586917,
                     'phase': 1.58941357},
  'q13': {'ec': 6.2867872105, 'ej': 25.1316550509, 'el': 6.9144529962},
  'q13_pulse_cos': { 'amp': 0.07763859,
                     'length': 39.

In [3]:
def comp_u(params, target_unitary):
    params = hk.data_structures.merge(evo.all_params, params)
    # evolve system on the eigen basis.
    sim_u = evo.eigen_basis(params)
    fidelity_vz, compensated_u = compute_fidelity_with_1q_rotation_axis(target_unitary,
                                                            sim_u,
                                                            opt_method=None)
    return compensated_u

compensated_u = comp_u(params, target_unitary)


In the `comp_u` function in the above cell, we begin by computing the result of the time evolution unitary, `sim_u`, for the given parameters in `params`. The computation involves determining the time-evolution unitary matrix in the eigenbasis of the idling Hamiltonian. Subsequently, we calculate a suitable virtual-Z compensation, to ensure that the resulting `compensated_u` closely resembles the matrix of simultaneous X gates.

We want to assess the level of correlation in the error. Let us define $U_\mathrm{error} = U_{\mathrm{compensated}}^{\dagger} U_{\mathrm{target}}$. By expanding $U_\mathrm{error}$ in the Pauli basis, we obtain $U_\mathrm{error} = \sum_{\vec{j}} a(\vec{j})P_{\vec{j}}$. In the following list, we will provide pairs of $(P_{\vec{j}}, |a(\vec{j})|^2)$. The values $|a(\vec{j})|^2$ can be interpreted as the probabilities of stochastic errors after performing Pauli twirling.
The colors of the text help visualize the weights of errors.

In [4]:
partition = [[0],[1],[2],[3],[4],[5]]
res = pauli_diagnose_partition(target_unitary, compensated_u, 20, partition, 'black')


IIIIII   9.9944e-01

IIIZIZ   1.2491e-04

IIIXIY   7.9677e-05

IIIYIY   3.5984e-05

ZIXIII   1.6914e-05

...

YIXXIZ   4.8337e-06

XZYIII   4.6177e-06

IZZZII   4.6046e-06

XIZYII   4.2224e-06

IYXZII   4.2033e-06

For optimization, let us use an object function based on the average gate fidelity with leakage. The formula is from Physical Review A 87, 022309 (2013).
$$
	C = 1 - \frac{1}{D(D + 1)}[Tr(PU^{\dagger}_{\text{sim}}P U_{\text{sim}}P)+|Tr(PU^{\dagger}_{\text{sim}} P U_{\text{target}})|^2],   
$$
where $P$ is the projector into the computational subspace $S$, and $D=2^n$ is the dimension of $S$.

In [5]:
def infidelity(params, target_unitary):
    params = hk.data_structures.merge(evo.all_params, params)
    # evolve system on the eigen basis.
    sim_u = evo.eigen_basis(params)
    # calculate fidelity
    fidelity_vz, _ = compute_fidelity_with_1q_rotation_axis(target_unitary,
                                                            sim_u,
                                                            opt_method=None)

    return jnp.log10(1 - fidelity_vz)


In [6]:
vg = jax.value_and_grad(infidelity)(params, target_unitary)
print('The infidelity of simultaneous X gates is ', 10**vg[0])


The infidelity of simultaneous X gates is  0.000550805312423797


In [7]:
print('The gradient of log10(infidelity):')
tree_print(vg[1])


The gradient of log10(infidelity):
{ 'capacitive_coupling_all_unify': {'strength': -0.5255497486236811},
  'inductive_coupling_all_unify': {'strength': -95.36296551953433},
  'q02': { 'ec': 3.6309946564779216,
           'ej': -1.1562086889388574,
           'el': 3.1335004633213988},
  'q02_pulse_cos': { 'amp': 0.09361627454710687,
                     'length': -0.011383359474712581,
                     'omega_d': 0.7530556302284802,
                     'phase': 0.05968693463255801},
  'q03': { 'ec': 0.9170256844994974,
           'ej': -0.27181664797136296,
           'el': 0.8245691473517796},
  'q03_pulse_cos': { 'amp': -0.2644388751359112,
                     'length': -0.0036655382182900955,
                     'omega_d': 0.516913680355986,
                     'phase': 0.02995326603035853},
  'q12': { 'ec': 0.6572720899711331,
           'ej': -0.1948206460831732,
           'el': 0.5607785992407223},
  'q12_pulse_cos': { 'amp': -0.9476511103827258,
                     'le

Show the optimization procedure of control parameters

In [8]:
adam_opt(infidelity,
         params,
         args=(target_unitary,),
         options={
             'adam_lr': 1e-6,
             'steps': 3
         })


{'adam_lr': 1e-06, 'adam_lr_decay_rate': 1000, 'steps': 3, 'adam_b1': 0.9, 'adam_b2': 0.999}
step: 0
parameters:
{ 'capacitive_coupling_all_unify': {'strength': 0.07225663},
  'inductive_coupling_all_unify': {'strength': -0.01256637},
  'q02': {'ec': 6.3532393823, 'ej': 25.1103087675, 'el': 6.344787656},
  'q02_pulse_cos': { 'amp': 0.07467672,
                     'length': 39.82459069,
                     'omega_d': 3.70771137,
                     'phase': 0.7558433},
  'q03': {'ec': 6.2550610547000005, 'ej': 25.1411401038, 'el': 5.000964059},
  'q03_pulse_cos': { 'amp': 0.06839676,
                     'length': 39.81302627,
                     'omega_d': 2.56897537,
                     'phase': 0.85390569},
  'q12': {'ec': 6.2233456874, 'ej': 25.1512697752, 'el': 5.6012793457},
  'q12_pulse_cos': { 'amp': 0.07142831,
                     'length': 39.85070846,
                     'omega_d': 3.07586917,
                     'phase': 1.58941357},
  'q13': {'ec': 6.2867872105, 'ej

{'capacitive_coupling_all_unify': {'strength': Array(0.07225963, dtype=float64)},
 'inductive_coupling_all_unify': {'strength': Array(-0.01256337, dtype=float64)},
 'q02': {'ec': Array(6.35323639, dtype=float64),
  'ej': Array(25.11031175, dtype=float64),
  'el': Array(6.34478467, dtype=float64)},
 'q02_pulse_cos': {'amp': Array(0.07467687, dtype=float64),
  'length': Array(39.82459369, dtype=float64),
  'omega_d': Array(3.70770889, dtype=float64),
  'phase': Array(0.75584081, dtype=float64)},
 'q03': {'ec': Array(6.25505806, dtype=float64),
  'ej': Array(25.1411431, dtype=float64),
  'el': Array(5.00096106, dtype=float64)},
 'q03_pulse_cos': {'amp': Array(0.06839683, dtype=float64),
  'length': Array(39.8130292, dtype=float64),
  'omega_d': Array(2.56897426, dtype=float64),
  'phase': Array(0.85390427, dtype=float64)},
 'q12': {'ec': Array(6.22334269, dtype=float64),
  'ej': Array(25.15127278, dtype=float64),
  'el': Array(5.60127635, dtype=float64)},
 'q12_pulse_cos': {'amp': Array(0